# Simulation de la ligne HERMES avec PyOptiX

In [1]:
import ctypes
import sys, os
sys.path.append("..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pandas as pd
from classes import Beamline, Source, OpticalElement, Diagram, RecordingMode, ToroidalMirror, PlaneHoloGrating
from exposed_functions import *
from ui_objects import scatter_plot_2d, show, plot_spd
from bokeh.io import push_notebook, output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [2]:
global optix
hc = h*c/nano/eV
try:
    test = optix
    print(test, "already initialized")
except NameError:
    optix = load_optix()
    print("OptiX library initialized")

intialzing SR library
OptiX library initialized


## Définition des optiques

In [3]:
Hermes = Beamline()
onduleur_BE = Source(element_type="GaussianSource", name="S_ONDUL_BE")
pupille = OpticalElement(element_type="PlaneFilm", name="pupille")
m1a = OpticalElement(element_type="PlaneMirror", name="M1A")
m1b = ToroidalMirror(element_type="ToroidalMirror", name="M1B")
m1c = ToroidalMirror(element_type="ToroidalMirror", name="M1C")
focalisation_horizontale = OpticalElement(element_type="PlaneFilm", name="foca")
f = OpticalElement(element_type="PlaneFilm", name="f")
reseau_450 = PlaneHoloGrating(element_type="PlaneHoloGrating", name="Reseau_450")
reseau_600 = OpticalElement(element_type="PlaneHoloGrating", name="Reseau_600")
reseau_200 = OpticalElement(element_type="PlaneHoloGrating", name="Reseau_200")
reseau_300 = OpticalElement(element_type="PlaneHoloGrating", name="Reseau_300")
reseau_MCA = OpticalElement(element_type="PlaneHoloGrating", name="Reseau_1800Multi")
m2 = OpticalElement(element_type="PlaneMirror", name="plan")
m3 = OpticalElement(element_type="ToroidalMirror", name="M3tor")
plan_intermediaire = OpticalElement(element_type="PlaneFilm", name="planint")
fente = OpticalElement(element_type="PlaneFilm", name="Fente")
m5 = OpticalElement(element_type="CylindricalMirror", name="M5CylConCave")
foc_intermediaire = OpticalElement(element_type="PlaneFilm", name="FocIntHo")
diagnostic = OpticalElement(element_type="PlaneFilm", name="Diag")
m6 = OpticalElement(element_type="ConicBaseCylindricalMirror", name="M6Cc")
m7 = OpticalElement(element_type="ConicBaseCylindricalMirror", name="M7Cc")
peem = OpticalElement(element_type="PlaneFilm", name="PEEM")

get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_parameter error b'No Error'
get_parameter error b'No Error'
get_parameter error b'No Error'
set_para

## Définition des paramètres des optiques

### Onduleur basse énergie

In [4]:
onduleur_BE.nrays = 5000
onduleur_BE.sigma_x = 1.74807e-4
onduleur_BE.sigma_y = 1.477385e-5
onduleur_BE.sigma_x_div = 3.17065e-3*degree # rad
onduleur_BE.sigma_y_div = 2.782436e-3*degree # rad
onduleur_BE.next = pupille
lambda_align = 6e-9
lambda_radiate = 6e-9

### Pupille

In [5]:
pupille.distance_from_previous = 20
set_recording(pupille.element_id, RecordingMode.recording_output)
pupille.next = m1a

### M1a 

In [6]:
m1a.distance_from_previous = 0.4783
m1a.theta = 2.5*degree # rad
m1a.phi = -90*degree # rad
m1a.next = m1b

### M1b 

In [7]:
m1b.distance_from_previous = 3.2096
m1b.theta = 2.5*degree # rad
m1b.phi = 180*degree # rad
m1b.minor_curvature = 1/1.802572 # m-1
m1b.major_curvature = 1/126.7433 # m-1
m1b.next = focalisation_horizontale

### M1c

In [8]:
m1b.distance_from_previous = 0.9791
m1b.theta = 1.2*degree # rad
m1b.phi = 180*degree # rad
m1b.minor_curvature = 1/0.8870043 # m-1
m1b.major_curvature = 1/229.5203 # m-1

### Plan de focalisation_horizontale

In [9]:
focalisation_horizontale.phi = -90*degree # rad
focalisation_horizontale.distance_from_previous = 3.2096
set_recording(pupille.element_id, RecordingMode.recording_output)
focalisation_horizontale.next = reseau_450

### Réseau 200 tpmm

In [10]:
reseau_200.line_density = 200/milli
reseau_200.inverse_distance1=0.1439452
reseau_200.inverse_distance2=0.1111111
reseau_200.elevation_angle1=np.arccos(0.7986355)
reseau_200.theta=0.1

set_parameter error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'


### Réseau 300 tpmm 

In [11]:
reseau_300.line_density = 300/milli
reseau_300.inverse_distance1=0.3247547
reseau_300.inverse_distance2=0.1111111
reseau_300.elevation_angle1=np.arccos(0.7986355)
reseau_300.theta=0.1

set_parameter error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'


### Reseau 450 tpmm

In [12]:
reseau_450.line_density = 450/milli
reseau_450.inverse_distance1=0.1761099
reseau_450.inverse_distance2=0.1111111
reseau_450.elevation_angle1=np.arccos(0.7986355)
reseau_450.theta=0.1

set_parameter error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'


### Reseau 600 tpmm

In [13]:
reseau_600.line_density = 600/milli
reseau_600.inverse_distance1=0.2002547
reseau_600.inverse_distance2=0.1111111
reseau_600.elevation_angle1=np.arccos(0.7986355)
reseau_600.theta=0.1

set_parameter error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'


### Reseau MCA

In [14]:
reseau_MCA.line_density = 1800/milli
reseau_MCA.inverse_distance1=1e-9
reseau_MCA.inverse_distance2=1e-7
reseau_MCA.elevation_angle1=np.arccos(0.939693)
reseau_MCA.theta=0.1
# Note : ce réseau n'est pas VLS

set_parameter error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'


## Définition des scripts d'alignement

### Onduleur

In [15]:
# *****     Données          **********
# dimension de la source électronique (sigmas en m)
#  SDM 169 X 6 µm ;    SDL 188 X 9 µm ;  SDC  321 x  8.1
Sh = 169e-6
Sv = 6e-6
# dimension de la source électronique (sigmas en rd)
#  SDM 27 X 5 µrd ;    SDL  30 x 4.5 µrd; SDC 14.6 x 4.5  µrd
Sph = 27e-6
Spv = 5e-6
# Longueur de l'onduleur (m)
Lond = 1.8
# Position (m) du centre de la S D au centre de l'onduleur
# Csd = dist SD / l'ond. utilisé comme origine (sens + vers l'aval)
# Cond = dist ond. / la SD utilisée comme origine (sens + vers l'aval)
Csd = 0
Cond = 1.849

# Facteur d'accord de l'onduleur   = augmentation de la divergence  en cas de desaccord
Ka = 1.4


# ************************************************
#      Elargissement du aux erreurs de pente
#
SSlV = 0
SSlH = 0
#SSlV = SlopeM1h * DistSM1 *  np.cos( M1dev /2.) + SlopeG1 * (DistSM1 + DistM1G ) )*2
# SSlH = $SlopeM1v *2 *  $DistSM1

#**************************************************


Lambda0 = lambda_radiate
print(f"Lambda = {lambda_radiate} ")
Su2 = Lambda0 * Lond / Ka/ (8 * pi**2) 
Spu2 = Lambda0 * Ka / (2 * Lond) 
print(f"SigmaU =  {np.sqrt(Su2)}  SigmaUp = {np.sqrt(Spu2)} ")
if  Cond == 0 :
    Zv = np.sqrt(Spv) / ( Spv**2 + Spu2)  * Csd 
    Zh = Sph**2 / ( Sph**2 + Spu2)  * Csd 
else :
    Zv = Spu2 / ( Spv**2 + Spu2)  * Cond 
    Zh = Spu2 / ( Sph**2 + Spu2)  * Cond 

print(f"Zv = {Zv}      Zh = {Zh} ")
# onduleur_BE.WaistZ = Zv
# onduleur_BE.WaistY = Zh
sqv2 = 1. / ( 1./Spv**2 + 1./Spu2 ) 
sqh2 = 1. / ( 1./Sph**2 + 1./Spu2 ) 
if  Cond == 0:
    SigmaV = np.sqrt(( Sv**2 + SSlV**2 + Su2 + Csd**2 *sqv2))
    SigmaH = np.sqrt(( Sh**2 + SSlH**2 + Su2 + Csd**2 *sqh2))
else:
    SigmaV = np.sqrt(( Sv**2 + SSlV**2 + Su2 + Cond**2 *sqv2))
    SigmaH = np.sqrt(( Sh**2 + SSlH**2 + Su2 + Cond**2 *sqh2))

print(f"sigmaY = {SigmaH}      sigmaZ = {SigmaV} ")
onduleur_BE.sigma_x = SigmaH
onduleur_BE.sigma_y =  SigmaV
SigmaPV = np.sqrt((Spv**2 + Spu2)) 
SigmaPH = np.sqrt((Sph**2 + Spu2)) 
print(f"sigmaYp = {SigmaPH}       sigmaZp = {SigmaPV}")
onduleur_BE.sigma_x_div = SigmaPH
onduleur_BE.sigma_y_div = SigmaPV

pupille.distance_from_previous += Cond - Csd # distance centre onduleur - centre source

Lambda = 6e-09 
SigmaU =  9.88446103441284e-06  SigmaUp = 4.8304589153964794e-05 
Zv = 1.829399293286219      Zh = 1.4088385762490476 
sigmaY = 0.00017480762475497748      sigmaZ = 1.4773846901636054e-05 
sigmaYp = 5.533835318595353e-05       sigmaZp = 4.856267428111155e-05


### Réseaux

In [16]:
cff = 0.6
omega = 0.72*degree # rad
delta_z = 15*milli # m
deviation = np.nan

if cff > 1:
    cff = 1/cff # selon definition
if focalisation_horizontale.next == reseau_MCA: # cas omega constant
    deviation = 2*np.arccos(lambda_align*reseau_MCA.line_density/2./np.sin(omega))
    alpha = (pi-deviation)/2 + omega
    beta =  (pi-deviation)/2 - omega
    reseau_MCA.theta = alpha
else: 
    reseau = focalisation_horizontale.next # cas cff constant
    nu = lambda_align*reseau.line_density
    k = 1-cff**2
    X = (np.sqrt(k**2 + nu**2*cff**2) - nu) / k
    alpha = np.arccos(X)
    beta = np.arccos(nu + X)
    deviation = pi - alpha - beta
    reseau.theta = alpha
    print(f"k={k}, nu={nu}, alpha={alpha}, beta={beta}")

    
m2.theta = (pi - deviation)/2
dist = delta_z/np.cos(deviation/2)*(1-np.sin(deviation/2)/2)
m2.distance_from_previous = dist

print(f"Alpha = {alpha/degree} deg,")
print(f"Beta = {beta/degree} deg")
print(f"Deviation = {deviation/degree} deg")
print(f"Distance reseau-M2 = {dist}")

set_parameter error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'
k=0.64, nu=0.0027, alpha=0.09185326919712176, beta=0.0550623166987319
Alpha = 5.262804659474084 deg,
Beta = 3.1548383570500538 deg
Deviation = 171.58235698347585 deg
Distance reseau-M2 = 0.1024669172245776


## Exécution de la simulation

In [17]:
align(onduleur_BE.element_id, lambda_align)
clear_impacts(onduleur_BE.element_id)

1

In [18]:
generate(onduleur_BE.element_id, lambda_radiate)

5000

In [19]:
radiate(onduleur_BE.element_id)

433612289

## Visualisations 

In [20]:
diagram = Diagram(ndim=5, nreserved=int(onduleur_BE.nrays))
get_spot_diagram(focalisation_horizontale.element_id, diagram, distance=20)
spots = pd.DataFrame(np.ctypeslib.as_array(diagram.spots, shape=(diagram.reserved, diagram.dim)),
                     columns=("X", "Y", "dX", "dY", "Lambda"))
print(spots.head())
figs = []
figs.append(plot_spd(spots, x_key="X", y_key="Y", light_plot=True, show_map=True))
figs.append(plot_spd(spots, x_key="X", y_key="dX", light_plot=False))
figs.append(plot_spd(spots, x_key="Y", y_key="dY", light_plot=True))

for fig in figs:
    show(fig)

          X         Y        dX        dY        Lambda
0 -0.002315 -0.000466 -0.000393  0.000002  6.000000e-09
1 -0.002590 -0.001844 -0.000436  0.000008  6.000000e-09
2  0.000735 -0.000300  0.000127  0.000001  6.000000e-09
3 -0.001117  0.000281 -0.000193 -0.000002  6.000000e-09
4  0.003476 -0.000637  0.000592  0.000003  6.000000e-09
